### Colab Activity 19.3: Implementing Funk SVD


**Expected Time = 60 minutes**



This activity focuses on using gradient descent to provide recommendations with collaborative filtering.  The purpose here is to get a high level introduction to the implementation of SVD Funk.  You will use the earlier ratings and a given user and item matrix to update the user factors.  In the next activity, you will implement the algorithms using `Surprise`.

### Index


- [Problem 1](#-Problem-1)
- [Problem 2](#-Problem-2)
- [Problem 3](#-Problem-3)
- [Problem 4](#-Problem-4)

In [1]:
import pandas as pd
import numpy as np

#### The Data

Below, the user reviews data is loaded as well as a $Q$ and $P$ matrix with some randomly built values from a similar process to the last activity.

In [14]:
reviews = pd.read_csv('data/user_rated.csv', index_col=0).iloc[:, :-2]
Q = pd.read_csv('data/Q.csv', index_col=0)
P = pd.read_csv('data/P.csv', index_col=0)
Q = Q[['F1', 'F2']]
P = P[['F1', 'F2']]

In [15]:
reviews.head()

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B
Alfred,3.0,4.0,NaN,4.0,4.0
Mandy,NaN,9.0,NaN,3.0,8.0
Lenny,2.0,5.0,8.0,9.0,NaN
Joan,3.0,NaN,9.0,4.0,9.0
Tino,1.0,1.0,NaN,9.0,5.0


In [4]:
Q.T.head() #item factors

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B
F1,-0.374013,0.073490,-0.046233,1.579452,0.281836
F2,0.158669,1.616899,0.150150,-0.426502,0.632869


In [5]:
P.head() #user factors

,F1,F2
Alfred,3.820956,3.395762
Mandy,3.710347,7.006197
Lenny,7.113263,3.952502
Joan,5.240167,10.035759
Tino,5.863280,2.197482


[Back to top](#-Index)

### Problem 1


#### Making Predictions

To make predictions you multiply a given row of $P$ by a column of $Q$.  Perform this operation for all users and items and assign a DataFrame of predicted values to `pred_df` below.  

HINT: For this step, use matrix multiplication rather than a nested loop. Matrix multiplication can be achieved using the `@` operator.

In [9]:

pred_df = P @ Q.T
# For example dot product of [F1, F2] in the Alfred row against [F1,F2] in the Michael Jackson column same for all combinations...


### ANSWER CHECK
pred_df

,Michael Jackson,Clint Black,Dropdead,Anti-Cimex,Cardi B
Alfred,-0.890284,5.771405,0.333221,4.586717,3.225954
Mandy,-0.276051,11.600983,0.880442,2.872159,5.479713
Lenny,-2.033312,6.913547,0.264603,9.549307,4.506187
Joan,-0.367525,16.611904,1.264603,3.996322,7.828187
Tino,-1.844270,3.983997,0.058877,8.323539,3.043199


### Problem 2


#### Measuring Error

Use your prediction for `Mandy` in terms of `Clint Black` to determine the error squared.  Assign this value to `ans2` below.

In [11]:
r_pred = 11.600983
r_actual = 9
ans2 = (r_pred - r_actual)**2

### ANSWER CHECK
print(ans2)

6.765112566288996


### Problem 3



#### Error for all Mandy Predictions

Now, compute the error squared for each of `Mandy`'s ratings where she had them -- `Clint Black`, `Anti-Cimex`, and `Cardi B`.  Assign these as a numpy array to `ans3`.

In [16]:

ans3 = [6.765112566288996, (2.872159 - 3)**2, (5.479713 - 8)**2]


### ANSWER CHECK
print(ans3)

[6.765112566288996, 0.016343321281000023, 6.351846562368999]


### Problem 4


#### Updating the Values

Now, perform the update for matrix $P$ based on the rule:

$$P_{a,b} := P_{a,b} - \alpha \sum_{j \in R_a}^N e_{a,j}Q_{b,j}$$

You will do this for the first factor of Mandy.  This means:

$$P_{1, 0} = -9.019710 - \alpha(e_{1, 1}Q_{1, 0} + e_{1, 3}Q_{3, 0} + e_{1, 4}Q_{4, 0})$$

Use $\alpha = 0.1$, and assign this new value as a float to `P_new`.

In [20]:
# Errors for Mandy's ratings
e_clint = r_pred - r_actual  # 11.600983 - 9 = 2.600983
e_anticimex = 2.872159 - 3   # -0.127841
e_cardib = 5.479713 - 8      # -2.520287

# Get Q values for the first factor (F1)
q_clint = Q.loc['Clint Black', 'F1']    # 0.073490
q_anticimex = Q.loc['Anti-Cimex', 'F1'] # 1.579452
q_cardib = Q.loc['Cardi B', 'F1']       # 0.281836

# Update P value
P_new = P.loc['Mandy', 'F1'] - 0.1 * (e_clint * q_clint + e_anticimex * q_anticimex + e_cardib * q_cardib)

### ANSWER CHECK
print(P_new)

3.7824552351265996


As an extra exercise, consider how to modularize this for each value of $P$.  Further, the update for $Q$ that occurs consistent with that of $P$ -- consider working through the full update process and modularizing the update process.